In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import os
import seaborn as sns
import tensorflow as tf
import os as os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as trans
import itertools
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: QtAgg


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
tf.config.list_physical_devices('GPU')

[]

In [6]:
gpus = tf.config.experimental.list_physical_devices('CPU')

In [7]:
gpus_local = tf.config.experimental.list_logical_devices('GPU')

In [8]:
len(gpus)

1

In [9]:
len(gpus_local)

0

In [10]:
data_dir = 'data'

In [11]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [12]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
             #os.remove("data")

In [13]:
data = tf.keras.utils.image_dataset_from_directory('data')

Found 442 files belonging to 2 classes.


In [14]:
data_iterator = data.as_numpy_iterator()

In [15]:
batch = data_iterator.next()

In [16]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [17]:
data = data.map(lambda x,y: (x/255, y))

In [18]:
data.as_numpy_iterator().next()

(array([[[[0.2720588 , 0.24803922, 0.1452206 ],
          [0.26102942, 0.23848039, 0.14166667],
          [0.23192401, 0.21182598, 0.12420343],
          ...,
          [0.24442402, 0.20465687, 0.06709559],
          [0.24963236, 0.20281863, 0.0646446 ],
          [0.24908088, 0.19859068, 0.0598652 ]],
 
         [[0.3071691 , 0.27677697, 0.1661152 ],
          [0.28694853, 0.2574755 , 0.15110295],
          [0.23180147, 0.20294118, 0.10147059],
          ...,
          [0.3204044 , 0.2740196 , 0.12493873],
          [0.3060049 , 0.25655636, 0.10441177],
          [0.29491422, 0.2463848 , 0.09148284]],
 
         [[0.36629903, 0.32120097, 0.20012255],
          [0.34178922, 0.29669118, 0.17561275],
          [0.25447303, 0.20845588, 0.08921569],
          ...,
          [0.42457107, 0.36378676, 0.1911152 ],
          [0.40098038, 0.34019607, 0.17058824],
          [0.38805148, 0.32726717, 0.1576593 ]],
 
         ...,
 
         [[0.89393383, 0.8498162 , 0.571875  ],
          [0.88020

In [19]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [20]:
val_size

2

In [21]:
train_size

9

In [22]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

In [24]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20
9/9 [==============================] - 10s 862ms/step - loss: 1.0635 - accuracy: 0.5382 - val_loss: 0.7457 - val_accuracy: 0.4688
Epoch 2/20
9/9 [==============================] - 9s 868ms/step - loss: 0.6528 - accuracy: 0.6493 - val_loss: 0.6778 - val_accuracy: 0.6094
Epoch 3/20
9/9 [==============================] - 9s 880ms/step - loss: 0.6533 - accuracy: 0.6493 - val_loss: 0.6762 - val_accuracy: 0.5781
Epoch 4/20
9/9 [==============================] - 8s 872ms/step - loss: 0.6619 - accuracy: 0.6111 - val_loss: 0.6502 - val_accuracy: 0.6562
Epoch 5/20
9/9 [==============================] - 8s 826ms/step - loss: 0.6604 - accuracy: 0.6250 - val_loss: 0.6489 - val_accuracy: 0.5938
Epoch 6/20
9/9 [==============================] - 8s 822ms/step - loss: 0.6670 - accuracy: 0.6736 - val_loss: 0.6530 - val_accuracy: 0.7656
Epoch 7/20
9/9 [==============================] - 8s 815ms/step - loss: 0.6550 - accuracy: 0.6389 - val_loss: 0.6481 - val_accuracy: 0.6875
Epoch 8/20
9/9 [===

In [27]:
#plot the training and validation accuracy and loss at each epoch
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [29]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [36]:
prediction_NN = model.predict(val_size)
prediction_NN = np.argmax(prediction_NN, axis=-1)
prediction_NN = le.inverse_transform(prediction_NN)

IndexError: tuple index out of range

In [35]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_size, prediction_NN)
print(cm)
sns.heatmap(cm, annot=True)

NameError: name 'prediction_NN' is not defined